In [9]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from googletrans import Translator 
pd.set_option('display.html.table_schema', True)

In [10]:
shop_df = pd.read_csv('../data/shops_trans.csv', encoding='utf-8')
shop_df = shop_df.drop(["shop_name"], axis=1)

In [11]:
date_df = pd.DataFrame(merged_sample["date"].str.split(".").tolist(),  columns = ['day','month','year'])

In [12]:
def split(name):
    sptname = name.split(' "')
    lastspt = sptname[0].split(" ")
    return lastspt[0]

shop_df["Region"] = list(map(split, shop_df.en_shop))

In [22]:
sales_df = pd.read_csv("../data/sales_train_v2.csv")
merged_sample = pd.merge(sales_df, shop_df, on = "shop_id")
date_df = pd.DataFrame(merged_sample["date"].str.split(".").tolist(),  columns = ['day','month','year'])
all_data = pd.merge(merged_sample, date_df, left_index=True, right_index=True)
del all_data["date_block_num"]
del all_data["date"]
all_data.head()

,shop_id,item_id,item_price,item_cnt_day,en_shop,Region,day,month,year
0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,02,01,2013
1,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,10,01,2013
2,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,04,01,2013
3,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,19,01,2013
4,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31,01,2013


In [60]:
categories_df = pd.read_csv("../data/item_categories_trans.csv")
del categories_df["item_category_name"]
categories_df.head()

,item_category_id,en_category
0,0,PC - Headsets / Headphones
1,1,Accessories - PS2
2,2,Accessories - PS3
3,3,Accessories - PS4
4,4,Accessories - PSP


In [59]:
item_df = pd.read_csv("../data/items.csv")
del item_df["item_name"]
item_df.head()

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


In [62]:
category_id = []
for i in all_data["item_id"]:
    category_id.append(np.squeeze(item_df["item_category_id"].loc[item_df["item_id"]==i]))
all_data["category_id"] = category_id
all_data.head()

,shop_id,item_id,item_price,item_cnt_day,en_shop,Region,day,month,year,category_id
0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,02,01,2013,37
1,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,10,01,2013,40
2,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,04,01,2013,5
3,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,19,01,2013,5
4,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31,01,2013,2


In [88]:
def splitcategory(x):
    if '-' in x:
        a = x.split('-')
        for i in a:
            i.strip()
        return a[0],a[1]
    else:
        return x, None

In [70]:
all_cate =  list(map(splitcategory, categories_df["en_category"]))
big_cate = []
small_cate = []
for i in all_cate:
    big, small = i
    big_cate.append(big)
    small_cate.append(small)
categories_df["big_cate"]= big_cate
categories_df["small_cate"] = small_cate

In [105]:
big_dict = {}
small_dict = {}
category_dict = {}
category_id = categories_df["item_category_id"].tolist()
category_name = categories_df["en_category"].tolist()
big_cate = categories_df["big_cate"].tolist()
small_cate = categories_df["small_cate"].tolist()
category = zip(category_name, big_cate, small_cate)
for i,j in zip(category_id, category):
    q,w,e = j
    category_dict[i] = q
    big_dict[i] = w
    small_dict[i] = e

In [107]:
category = []
big = []
small = []
for i in all_data["category_id"]:
    category.append(category_dict[i])
    big.append(big_dict[i])
    small.append(small_dict[i])
all_data["category"] = category
all_data["big_cate"] = big
all_data["small_cate"] = small
all_data.head()

,shop_id,item_id,item_price,item_cnt_day,en_shop,Region,day,month,year,category_id,category,big_cate,small_cate
0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,02,01,2013,37,Cinema - Blu-Ray,Cinema,Blu
1,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,10,01,2013,40,Cinema - DVD,Cinema,DVD
2,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,04,01,2013,5,Accessories - PSVita,Accessories,PSVita
3,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,19,01,2013,5,Accessories - PSVita,Accessories,PSVita
4,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31,01,2013,2,Accessories - PS3,Accessories,PS3


In [109]:
all_data.shape

(2935849, 13)

In [108]:
all_data.to_csv("merged_data.csv", index=False)